In [3]:
# --- Generalized ANOVA (one-way or two-way) with Tukey–Kramer ---
# pip install pandas statsmodels scipy
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import combinations
from itertools import combinations
from collections import OrderedDict

# Try to get critical value q via statsmodels or scipy (fallback)
def _get_qcrit(prob, k, df_error):
    try:
        from statsmodels.stats.libqsturng import qsturng
        return qsturng(prob, k, df_error)
    except Exception:
        try:
            from scipy.stats import studentized_range
            return studentized_range.ppf(prob, k, df_error)
        except Exception:
            raise ImportError("Need statsmodels.libqsturng or scipy.stats.studentized_range for Tukey–Kramer.")

# ---------------- CONFIG ----------------
USE_EXCEL = True
file_path = r"C:\Users\300393449\OneDrive - Douglas College\Documents\4th Semester\2_Business_Statistics_II\Python\DataScratch.xlsx"
sheet_name = "11_ANOVA"

# Optional pretty display names for factors in messages (None keeps x1/x2)
PRETTY = {
    "x1": None,  # e.g., "diet"
    "x2": None,  # e.g., "reporting status"
}

# If you want to explicitly tell the loader what your group columns are in wide format, set this list.
# Leave as None to auto-detect numeric columns (safer if the sheet only contains the wide table).
WIDE_GROUP_COLS = None  # e.g., ["50-59","60-69","70+"]

# -------------- LOAD & DETECT --------------
def detect_format(df):
    cols = set(df.columns.str.strip())
    has_y = 'y' in cols
    has_x1 = 'x1' in cols
    has_x2 = 'x2' in cols
    if has_y and has_x1:
        return 'long_two' if has_x2 else 'long_one'
    # Otherwise assume wide if 2+ numeric cols (optionally filtered by WIDE_GROUP_COLS)
    if WIDE_GROUP_COLS:
        return 'wide'
    numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    return 'wide' if len(numeric_cols) >= 2 else 'unknown'

def load_long(df):
    # Coerce x1/x2 to categorical if they look categorical (non-numeric or few uniques)
    out = df.copy()
    for fac in ['x1','x2']:
        if fac in out.columns:
            if not pd.api.types.is_numeric_dtype(out[fac]) or out[fac].nunique() <= 12:
                out[fac] = out[fac].astype('category')
    return out[['y','x1'] + (['x2'] if 'x2' in out.columns else [])].dropna()

def load_from_wide(df):
    if WIDE_GROUP_COLS:
        wide = df[WIDE_GROUP_COLS].copy()
    else:
        # Keep only numeric columns (assumed to be group columns).
        wide = df.select_dtypes(include=[np.number]).copy()
    # Drop rows that are all NaN across groups
    wide = wide.dropna(how='all')
    long_df = wide.melt(var_name='x1', value_name='y').dropna()
    # make group labels strings
    long_df['x1'] = long_df['x1'].astype(str).astype('category')
    return long_df

def get_alpha(default=0.05):
    try:
        s = input(f"Enter significance level alpha (e.g., 0.05) [default {default}]: ").strip()
        if not s: return default
        a = float(s)
        return a if 0 < a < 1 else default
    except Exception:
        return default

def _fname(f):  # pretty name for factor if provided
    return PRETTY.get(f) or f

def print_hypotheses_oneway(k):
    print("\n--- Hypotheses (One-way ANOVA) ---")
    print(f"H0: μ1 = μ2 = ... = μ{k}")
    print("H1: Not all means are equal.\n")

def print_hypotheses_twoway(fa, fb):
    print("\n--- Hypotheses (Two-way ANOVA with interaction) ---")
    print(f"Main effect {fa}: H0 = no mean difference across {fa} levels")
    print(f"Main effect {fb}: H0 = no mean difference across {fb} levels")
    print(f"Interaction {fa}×{fb}: H0 = no interaction effect\n")

# -------------- ANALYSIS --------------
def one_way_anova(df_long, alpha=0.05):
    model = ols("y ~ C(x1)", data=df_long).fit()
    anova_tbl = sm.stats.anova_lm(model, typ=2)
    print("--- One-way ANOVA table ---")
    print(anova_tbl.to_string())
    F = anova_tbl.loc["C(x1)", "F"]
    p = anova_tbl.loc["C(x1)", "PR(>F)"]
    df1 = int(anova_tbl.loc["C(x1)", "df"])
    df2 = int(anova_tbl.loc["Residual", "df"])
    print(f"\nF({df1}, {df2}) = {F:.4f}, p-value = {p:.6g}")
    print(f"Decision at α={alpha}: {'REJECT H0' if p < alpha else 'Fail to reject H0'}\n")
    return model, anova_tbl

def tukey_kramer(df_long, alpha=0.05, model=None):
    print("--- Tukey–Kramer (Tukey HSD) ---")
    tk = pairwise_tukeyhsd(endog=df_long['y'], groups=df_long['x1'], alpha=alpha)
    print(tk.summary())

    # Compute CR_{i,j} explicitly
    if model is None:
        model = ols("y ~ C(x1)", data=df_long).fit()
    mse = model.mse_resid
    df_error = int(model.df_resid)
    groups = df_long['x1'].cat.categories if hasattr(df_long['x1'], 'cat') else df_long['x1'].unique()
    k = len(groups)
    qcrit = _get_qcrit(1 - alpha, k, df_error)

    means = df_long.groupby('x1')['y'].mean()
    ns = df_long.groupby('x1')['y'].size()

    rows = []
    for a, b in combinations(means.index, 2):
        diff = abs(means[a] - means[b])
        CR = qcrit * np.sqrt(mse / 2 * (1/ns[a] + 1/ns[b]))
        conclusion = "DIFFERENT" if diff > CR else "not different"
        rows.append({"pair": f"{a} vs {b}",
                     "mean(a)": means[a], "mean(b)": means[b],
                     "|mean diff|": diff, "CR_{i,j}": CR, "Conclusion": conclusion})
    comp = pd.DataFrame(rows, columns=["pair","mean(a)","mean(b)","|mean diff|","CR_{i,j}","Conclusion"])
    print("\n--- Tukey–Kramer critical range comparison ---")
    print(comp.to_string(index=False))
    print("\nRule: If |mean_i − mean_j| > CR_{i,j}, conclude those means are DIFFERENT.\n")
    return comp

def two_way_anova(df_long, alpha=0.05):
    # x1, x2 should be categorical
    for fac in ['x1','x2']:
        if fac in df_long and (not pd.api.types.is_categorical_dtype(df_long[fac])):
            if not pd.api.types.is_numeric_dtype(df_long[fac]) or df_long[fac].nunique() <= 12:
                df_long[fac] = df_long[fac].astype('category')

    model = ols("y ~ C(x1) + C(x2) + C(x1):C(x2)", data=df_long).fit()
    anova_tbl = sm.stats.anova_lm(model, typ=2)
    print("--- Two-way ANOVA table (Type II) ---")
    print(anova_tbl.to_string())

    for term in ["C(x1)","C(x2)","C(x1):C(x2)"]:
        if term in anova_tbl.index:
            p = anova_tbl.loc[term, "PR(>F)"]
            print(f"{term} p-value = {p:.6g} -> {'REJECT' if p < alpha else 'Fail to reject'} at α={alpha}")
    print()
    return model, anova_tbl

# ----- Assignment-style ANOVA table (Between/Within/Total) -----
def _round0(x):
    """Round to nearest whole number and return int."""
    return int(np.rint(float(x)))

def assignment_style_anova_table(anova_tbl, include_total_ms=True, round_to_int=True):
    """
    Build 'Between / Within / Total' table from a statsmodels ANOVA table.
    Works for one-way and two-way: 'Between' = sum of all model terms (non-Residual).
    """
    if "Residual" not in anova_tbl.index:
        raise ValueError("ANOVA table missing 'Residual' row.")

    mask_between = anova_tbl.index != "Residual"
    ss_between = anova_tbl.loc[mask_between, "sum_sq"].sum()
    df_between = anova_tbl.loc[mask_between, "df"].sum()

    ss_within = anova_tbl.loc["Residual", "sum_sq"]
    df_within = anova_tbl.loc["Residual", "df"]

    ss_total = ss_between + ss_within
    df_total = df_between + df_within

    ms_between = ss_between / df_between if df_between > 0 else np.nan
    ms_within  = ss_within  / df_within  if df_within  > 0 else np.nan
    ms_total   = ss_total   / df_total   if (include_total_ms and df_total > 0) else np.nan

    if round_to_int:
        ss_between_r = _round0(ss_between); ss_within_r  = _round0(ss_within);  ss_total_r   = _round0(ss_total)
        ms_between_r = _round0(ms_between); ms_within_r  = _round0(ms_within);  ms_total_r   = _round0(ms_total) if include_total_ms else ""
        df_between_r = int(df_between);     df_within_r  = int(df_within);      df_total_r   = int(df_total)
    else:
        ss_between_r = ss_between; ss_within_r = ss_within; ss_total_r = ss_total
        ms_between_r = ms_between; ms_within_r = ms_within; ms_total_r = ms_total if include_total_ms else ""
        df_between_r = df_between; df_within_r = df_within; df_total_r = df_total

    table = pd.DataFrame({
        "Sum of Squares":        [ss_between_r, ss_within_r, ss_total_r],
        "Degrees of Freedom":    [df_between_r, df_within_r, df_total_r],
        "Mean Sum of Squares":   [ms_between_r, ms_within_r, ms_total_r],
    }, index=["Between", "Within", "Total"])
    return table

def print_assignment_style_anova_table(anova_tbl, include_total_ms=True, round_to_int=False, decimals=3):
    """
    Pretty-print the Between/Within/Total ANOVA table.
    - round_to_int=False keeps raw (unrounded) numbers in the DataFrame.
    - decimals controls how many decimal places are SHOWN when printing.
    """
    table = assignment_style_anova_table(anova_tbl, include_total_ms, round_to_int)
    table["Degrees of Freedom"] = table["Degrees of Freedom"].astype(int)

    def _fmt(x):
        try:
            if x == "" or x is None:
                return ""
            return f"{float(x):.{decimals}f}"
        except Exception:
            return x

    print(f"\nComplete the ANOVA summary table below. (Showing {decimals} decimal places)")
    print(table.to_string(formatters={
        "Sum of Squares": _fmt,
        "Mean Sum of Squares": _fmt
    }))
    print()

# ----- Two-way: explicit interaction summary -----
def print_interaction_test(anova_tbl, term="C(x1):C(x2)", decimals=3, p_decimals=6):
    if term not in anova_tbl.index:
        print(f"Interaction term '{term}' not found in ANOVA table.")
        return
    df1 = int(anova_tbl.loc[term, "df"]); df2 = int(anova_tbl.loc["Residual", "df"])
    ss  = float(anova_tbl.loc[term, "sum_sq"])
    ms  = ss / df1 if df1 > 0 else np.nan
    F   = float(anova_tbl.loc[term, "F"])
    p   = float(anova_tbl.loc[term, "PR(>F)"])
    term_pretty = term.replace("C(", "").replace(")", "")
    print(f"\nInteraction test ({term_pretty}):")
    print(
        f"  SS = {ss:.{decimals}f}, df = {df1}, MS = {ms:.{decimals}f}\n"
        f"  F({df1}, {df2}) = {F:.{decimals}f}, p = {p:.{p_decimals}g}"
    )

# ----- Factor-agnostic mapping + MCQ conclusions -----
def factor_level_mapping(df, factor_cols=("x1","x2","x3","x4")):
    mapping = {}
    for col in factor_cols:
        if col in df.columns:
            if pd.api.types.is_categorical_dtype(df[col]):
                levels = list(df[col].cat.categories)
            else:
                levels = sorted(map(str, pd.Series(df[col].unique()).dropna().tolist()))
            items = [f"item{i+1}" for i in range(len(levels))]
            itl = OrderedDict(zip(items, levels))
            lti = {lvl: itm for itm, lvl in itl.items()}
            mapping[col] = {"levels": levels, "item_to_level": itl, "level_to_item": lti}
    return mapping

def print_factor_level_mapping(mapping):
    if not mapping:
        return
    print("\n--- Factor level mapping (generic → original) ---")
    for fac, m in mapping.items():
        pairs = [f"{item} \u2192 {lvl}" for item, lvl in m["item_to_level"].items()]
        print(f"{fac} level map: " + "; ".join(pairs))

def conclude_main_effect_generic(anova_tbl, factor="x2", inter=("x1","x2"), alpha=0.05):
    """
    MCQ-style conclusion for main effect of `factor`, handling interaction.
    Returns (letter, message) with options:
      B: Do not reject (insufficient evidence)
      C: Reject (sufficient evidence that not all means are equal)
      D: Inappropriate (interaction significant)
    """
    term_factor = f"C({factor})"
    term_inter  = f"C({inter[0]}):C({inter[1]})"

    if term_inter in anova_tbl.index:
        p_int = float(anova_tbl.loc[term_inter, "PR(>F)"])
        if p_int < alpha:
            return ("D",
                    f"It is inappropriate to analyze because {_fname(inter[0])} and {_fname(inter[1])} interact "
                    f"(interaction p = {p_int:.4g} < {alpha}).")

    p = float(anova_tbl.loc[term_factor, "PR(>F)"])
    if p < alpha:
        return ("C",
                f"Reject the null hypothesis for the main effect of {_fname(factor)} "
                f"(p = {p:.4g} < {alpha}). There is sufficient evidence that not all "
                f"{_fname(factor)} means are equal.")
    else:
        return ("B",
                f"Do not reject the null hypothesis for the main effect of {_fname(factor)} "
                f"(p = {p:.4g} \u2265 {alpha}). There is insufficient evidence to conclude that "
                f"not all {_fname(factor)} means are equal.")

def tukey_kramer_for_factor(df_long, model, factor="x1", y_col="y", alpha=0.05):
    """
    Tukey–Kramer pairwise test for any single factor (x1/x2/...), using residual MSE and df from `model`.
    Returns a DataFrame with both item- and original-level labels.
    """
    if factor not in df_long.columns:
        raise ValueError(f"Factor {factor} not found in df.")
    mse = float(model.mse_resid)
    df_error = int(model.df_resid)
    means = df_long.groupby(factor)[y_col].mean()
    ns    = df_long.groupby(factor)[y_col].size()
    levels = list(means.index)
    k = len(levels)
    qcrit = _get_qcrit(1 - alpha, k, df_error)

    mapping = factor_level_mapping(df_long, (factor,)).get(factor)
    level_to_item = mapping["level_to_item"] if mapping else {lvl: str(lvl) for lvl in levels}

    rows = []
    for a, b in combinations(levels, 2):
        diff = abs(means[a] - means[b])
        CR = qcrit * np.sqrt(mse/2 * (1/ns[a] + 1/ns[b]))
        rows.append({
            "pair_items": f"{level_to_item.get(a, a)} vs {level_to_item.get(b, b)}",
            "pair_levels": f"{a} vs {b}",
            "|mean diff|": diff,
            "CR_{i,j}": CR,
            "significant": bool(diff > CR)
        })
    return pd.DataFrame(rows)

def conclude_pairs_mcq(df_long, model, anova_tbl, alpha=0.05, factor="x1",
                       target_pairs_items=("item1","item2","item3")):
    """
    Prints sentence-style answers (no letters) for pair questions on `factor`.
    Logic:
      - If interaction is significant -> comparisons unwarranted (explain why).
      - Else if main effect of factor NOT significant -> comparisons unwarranted (explain why).
      - Else run Tukey–Kramer and answer:
          "Yes — the means differ significantly..." or
          "No — insufficient evidence that the means differ..."
      Also prints a generic item → original level mapping and a small TK table.
    """
    from collections import OrderedDict

    other = "x2" if factor == "x1" else "x1"
    term_inter = f"C({factor}):C({other})"
    p_inter = float(anova_tbl.loc[term_inter, "PR(>F)"]) if term_inter in anova_tbl.index else 1.0

    # Build mapping once for readable pair labels
    mapping_all = factor_level_mapping(df_long, (factor,))
    item_to_level = mapping_all.get(factor, {}).get("item_to_level", OrderedDict())

    # 1) Interaction check
    if p_inter < alpha:
        print(f"\nPairwise comparisons for {_fname(factor)} are unwarranted because "
              f"{_fname(factor)} and {_fname(other)} interact (p = {p_inter:.4g} < {alpha}).")
        for i in range(1, min(3, len(item_to_level))):
            iA, iB = 'item1', f'item{i+1}'
            print(f"Are the means for {iA} and {iB} significantly different? "
                  f"Unwarranted due to significant interaction between {_fname(factor)} and {_fname(other)}.")
        return

    # 2) Main-effect check
    term_factor = f"C({factor})"
    p_fac = float(anova_tbl.loc[term_factor, "PR(>F)"]) if term_factor in anova_tbl.index else 1.0
    if p_fac >= alpha:
        print(f"\nPairwise comparisons for {_fname(factor)} are unwarranted because the main effect of "
              f"{_fname(factor)} is not significant (p = {p_fac:.4g} \u2265 {alpha}).")
        for i in range(1, min(3, len(item_to_level))):
            iA, iB = 'item1', f'item{i+1}'
            print(f"Are the means for {iA} and {iB} significantly different? "
                  f"Unwarranted because the main effect of {_fname(factor)} is not significant.")
        return

    # 3) Warranted: run Tukey–Kramer on this factor
    tk = tukey_kramer_for_factor(df_long, model, factor=factor, y_col="y", alpha=alpha)

    # Show mapping and TK table
    print_factor_level_mapping(mapping_all)
    print(f"\nTukey–Kramer pairwise results for {_fname(factor)} (α = {alpha:.3f})")

    def _fmt3(x):
        try: return f"{float(x):.3f}"
        except Exception: return x

    if not tk.empty:
        print(tk.to_string(index=False, formatters={"|mean diff|": _fmt3, "CR_{i,j}": _fmt3}))

    # Helper to fetch a row by item pair
    def _row_for_items(iA, iB):
        pairA = f"{iA} vs {iB}"
        pairB = f"{iB} vs {iA}"
        hit = tk[tk["pair_items"].isin([pairA, pairB])]
        return None if hit.empty else hit.iloc[0]

    # Answer for item1 vs item2 and item1 vs item3 (if present)
    for i in range(1, min(3, len(item_to_level))):
        iA, iB = "item1", f"item{i+1}"
        row = _row_for_items(iA, iB)
        if row is None:
            print(f"\nAre the means for {iA} and {iB} significantly different? Not applicable (pair not found).")
            continue

        lvlA = item_to_level.get(iA, iA)
        lvlB = item_to_level.get(iB, iB)
        diff = float(row["|mean diff|"])
        CR   = float(row["CR_{i,j}"])
        if bool(row["significant"]):
            print(f"\nAre the means for {iA} and {iB} significantly different? "
                  f"Yes — the means for {iA} ({lvlA}) and {iB} ({lvlB}) differ significantly at α={alpha} "
                  f"( |mean diff| = {diff:.3f} > CR = {CR:.3f} ).")
        else:
            print(f"\nAre the means for {iA} and {iB} significantly different? "
                  f"No — there is insufficient evidence that the means for {iA} ({lvlA}) and {iB} ({lvlB}) differ "
                  f"at α={alpha} ( |mean diff| = {diff:.3f} ≤ CR = {CR:.3f} ).")

# -------------- MAIN --------------
if __name__ == "__main__":
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    fmt = detect_format(df)

    if fmt.startswith('long'):
        df_long = load_long(df)
        has_x2 = 'x2' in df_long.columns
        choice = input(f"ANOVA type? (1=One-way, 2=Two-way) [default {'2' if has_x2 else '1'}]: ").strip()
        do_two = (choice == "2") or (choice == "" and has_x2)
    elif fmt == 'wide':
        df_long = load_from_wide(df)
        has_x2 = False
        print("Detected WIDE format: melted to long as columns (x1=group, y=response).")
        do_two = False
    else:
        raise ValueError("Could not detect data format. Use long format with columns y,x1[,x2] or a wide table of group columns.")

    alpha = get_alpha(0.05)

    # Hypotheses + Run
    if not do_two:
        k = df_long['x1'].nunique()
        print_hypotheses_oneway(k)
        model, anova_tbl = one_way_anova(df_long, alpha=alpha)
        # Assignment-style ANOVA summary for one-way
        print_assignment_style_anova_table(anova_tbl, include_total_ms=True, round_to_int=False, decimals=3)
        # Tukey for one-way
        tukey_kramer(df_long, alpha=alpha, model=model)

    else:
        print_hypotheses_twoway('x1','x2')
        model, anova_tbl = two_way_anova(df_long, alpha=alpha)

        # Assignment-style ANOVA summary for two-way
        print_assignment_style_anova_table(anova_tbl, include_total_ms=True, round_to_int=False, decimals=3)

        # Explicit interaction summary
        print_interaction_test(anova_tbl, term="C(x1):C(x2)", decimals=3, p_decimals=6)

        # Show item# → level mapping for both factors
        mapping = factor_level_mapping(df_long, ("x1","x2"))
        print_factor_level_mapping(mapping)

        # MCQ conclusion for the main effect of x2 (generic; uses PRETTY names if set)
        _, msg = conclude_main_effect_generic(anova_tbl, factor="x2", inter=("x1","x2"), alpha=alpha)
        print(f"\nEffect of {_fname('x2')} (main effect): {msg}")


        # If warranted, pairwise on x1 (item1 vs item2, item1 vs item3)
        conclude_pairs_mcq(df_long, model, anova_tbl, alpha=alpha, factor="x1",
                           target_pairs_items=("item1","item2","item3"))


Detected WIDE format: melted to long as columns (x1=group, y=response).


Enter significance level alpha (e.g., 0.05) [default 0.05]:  .05



--- Hypotheses (One-way ANOVA) ---
H0: μ1 = μ2 = ... = μ4
H1: Not all means are equal.

--- One-way ANOVA table ---
             sum_sq    df          F        PR(>F)
C(x1)     17.067255   3.0  22.022683  6.349533e-08
Residual   8.266510  32.0        NaN           NaN

F(3, 32) = 22.0227, p-value = 6.34953e-08
Decision at α=0.05: REJECT H0


Complete the ANOVA summary table below. (Showing 3 decimal places)
        Sum of Squares  Degrees of Freedom Mean Sum of Squares
Between         17.067                   3               5.689
Within           8.267                  32               0.258
Total           25.334                  35               0.724

--- Tukey–Kramer (Tukey HSD) ---
        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
      group1         group2  meandiff p-adj   lower   upper  reject
-------------------------------------------------------------------
Filling Time (Sec)   Mold 60   -1.434    0.0 -2.0832 -0.7848   True
Filling Time (Sec) Mold 72.5  

C:\Users\300393449\AppData\Local\Temp\ipykernel_25956\3512384346.py:127: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means = df_long.groupby('x1')['y'].mean()
C:\Users\300393449\AppData\Local\Temp\ipykernel_25956\3512384346.py:128: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ns = df_long.groupby('x1')['y'].size()


For 2 way, refer to Final Excel wWorking file